In [ ]:
import os
from pathlib import Path
import json
import webbrowser
import requests
import pandas as pd
import numpy as np
import time
import logging
import tkinter as tk
from tkinter import filedialog, ttk, messagebox
from http.server import BaseHTTPRequestHandler, HTTPServer
import urllib.parse
from dotenv import load_dotenv
import openai
from functools import lru_cache
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime, timedelta, timezone

# Setup logging
logging.basicConfig(level=logging.DEBUG)  # Set to DEBUG
logger = logging.getLogger(__name__)
logging.getLogger('urllib3').setLevel(logging.INFO)
logging.getLogger('httpcore').setLevel(logging.INFO)
logging.getLogger('httpx').setLevel(logging.INFO)
logging.getLogger('openai').setLevel(logging.INFO)

# Global flag for debug logging (enabled)
debug_logging = True

def enable_debug_logging():
    global debug_logging
    debug_logging = True
    logging.getLogger().setLevel(logging.DEBUG)

# Load environment variables
logger.info("Attempting to load keys.env...")
try:
    with open('keys.env', 'r', encoding='utf-8') as f:
        raw_content = f.read().strip()
    logger.info(f"Raw contents of keys.env loaded successfully")
except Exception as e:
    logger.error(f"Failed to read keys.env: {str(e)}")

load_dotenv('keys.env')
logger.info(f"Loaded .env from: {os.getenv('DOTENV_PATH', 'keys.env')}")
logger.info(f"DIGIKEY_CLIENT_ID: {'Set' if os.environ.get('DIGIKEY_CLIENT_ID') else 'Not set'}")
logger.info(f"DIGIKEY_CLIENT_SECRET: {'Set' if os.environ.get('DIGIKEY_CLIENT_SECRET') else 'Not set'}")
logger.info(f"MOUSER_API_KEY: {'Set' if os.environ.get('MOUSER_API_KEY') else 'Not set'}")
logger.info(f"CHATGPT_API_KEY: {'Set' if os.environ.get('CHATGPT_API_KEY') else 'Not set'}")

# Configure OpenAI API
openai.api_key = os.environ.get('CHATGPT_API_KEY')

@lru_cache(maxsize=100)  # Cache ChatGPT responses
def call_chatgpt(prompt):
    try:
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a supply chain analysis expert with access to historical market data."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=1000
        )
        return response.choices[0].message.content
    except Exception as e:
        logger.error(f"ChatGPT API error: {str(e)}")
        return None

class OAuthHandler(BaseHTTPRequestHandler):
    def do_GET(self):
        query = urllib.parse.urlparse(self.path).query
        code = urllib.parse.parse_qs(query).get('code', [None])[0]
        self.server.auth_code = code
        self.send_response(200)
        self.end_headers()
        self.wfile.write(b"Auth code received. You can close this window.")

class BOMAnalyzerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("NPI BOM Analyzer")
        self.root.geometry("1200x700")
        self.root.minsize(800, 500)
        self.root.resizable(True, True)
        self.root.configure(bg='white')
        logger.info("Initializing GUI...")

        self.style = ttk.Style()
        self.style.theme_use('clam')

        self.main_frame = ttk.Frame(self.root)
        self.main_frame.pack(fill="both", expand=True, padx=10, pady=10)
        self.main_frame.grid_rowconfigure(0, weight=1)
        self.main_frame.grid_columnconfigure(0, weight=2, minsize=300)
        self.main_frame.grid_columnconfigure(1, weight=3)

        self.config_frame = ttk.Frame(self.main_frame, padding=10, relief="groove", width=300)
        self.config_frame.grid(row=0, column=0, sticky="nsew", padx=(0, 10))

        self.config_title = ttk.Label(self.config_frame, text="Sweet Spot Configuration", font=("TkDefaultFont", 12, "bold"))
        self.config_title.pack(fill="x", pady=(0, 10))

        config_entries = [
            ("Total Units to Build:", "total_units", "1", "Number of finished goods to build"),
            ("Max Cost Premium (%):", "max_premium", "15", "Max % above cheapest price"),
            ("Target Lead Time (weeks):", "target_lead_time", "8", "Max acceptable lead time"),
            ("Cost Weight (0-1):", "cost_weight", "0.5", "Priority of cost in sweet spot"),
            ("Lead Time Weight (0-1):", "lead_time_weight", "0.5", "Priority of lead time in sweet spot"),
        ]
        
        for label, attr, default, hint in config_entries:
            ttk.Label(self.config_frame, text=label).pack(fill="x", pady=(5, 0))
            entry = ttk.Entry(self.config_frame, width=20)
            entry.pack(fill="x", pady=(0, 2))
            entry.insert(0, default)
            ttk.Label(self.config_frame, text=hint, font=("Arial", 8), wraplength=250).pack(fill="x", pady=(0, 5))
            setattr(self, f"{attr}_entry", entry)

        self.tariff_frame = ttk.LabelFrame(self.config_frame, text="Custom Tariff Rates (%)", padding=5)
        self.tariff_frame.pack(fill="x", pady=(5, 0))
        self.tariff_entries = {}
        top_countries = ["China", "Mexico", "India", "Vietnam", "Turkey"]
        for country in top_countries:
            frame = ttk.Frame(self.tariff_frame)
            frame.pack(fill="x", pady=(0, 2))
            ttk.Label(frame, text=f"{country}: ").pack(side="left", padx=(0, 2))
            entry = ttk.Entry(frame, width=5)
            entry.insert(0, "")  # Default to blank (uses USITC or predictive rate)
            entry.pack(side="left")
            self.tariff_entries[country] = entry
        ttk.Label(self.tariff_frame, text="(Leave blank for default from USITC; confirm with suppliers)", font=("Arial", 8)).pack(fill="x", pady=(2, 0))

        self.validation_label = ttk.Label(self.config_frame, text="", foreground="red")
        self.validation_label.pack(fill="x", pady=5)

        self.main_content_frame = ttk.Frame(self.main_frame)
        self.main_content_frame.grid(row=0, column=1, sticky="nsew")
        self.main_content_frame.grid_rowconfigure(1, weight=1)
        self.main_content_frame.grid_columnconfigure(0, weight=1)

        self.controls_frame = tk.Frame(self.main_content_frame, background='white')
        self.controls_frame.grid(row=0, column=0, sticky="ew", pady=(0, 5))
        self.controls_frame.grid_columnconfigure(0, weight=1)

        self.file_label = ttk.Label(self.controls_frame, text="No BOM loaded")
        self.file_label.grid(row=0, column=0, padx=5, sticky="w")
        
        self.load_button = ttk.Button(self.controls_frame, text="Load BOM CSV", command=self.load_bom)
        self.load_button.grid(row=0, column=1, padx=5)
        
        self.run_button = ttk.Button(self.controls_frame, text="Run Analysis", command=self.validate_and_run, state="disabled")
        self.run_button.grid(row=0, column=2, padx=5)
        
        self.ai_summary_button = ttk.Button(self.controls_frame, text="AI Summary", command=self.generate_ai_summary, state="disabled")
        self.ai_summary_button.grid(row=0, column=3, padx=5)
        
        self.status = tk.Label(self.controls_frame, text="Ready", foreground="black", background="lightgray")
        self.status.grid(row=0, column=4, padx=5, sticky="w")
        
        self.rate_label = ttk.Label(self.controls_frame, text="Rate Limits - DigiKey: 120/120, Mouser: 1000/1000")
        self.rate_label.grid(row=0, column=5, padx=5, sticky="e")
        
        self.progress = ttk.Progressbar(self.controls_frame, orient="horizontal", length=200, mode="determinate")
        self.progress_label = ttk.Label(self.controls_frame, text="Progress: 0%", wraplength=200)
        self.progress_label.grid(row=1, column=0, columnspan=6, padx=5, pady=2, sticky="w")

        self.results_notebook = ttk.Notebook(self.main_content_frame)
        self.results_notebook.grid(row=2, column=0, sticky="nsew")

        self.tree_frame = ttk.Frame(self.results_notebook)
        self.results_notebook.add(self.tree_frame, text="Parts List")
        
        columns = [
            "PartNumber", "QtyNeeded", "Part", "Price", "MinOrderQty",
            "TariffCode", "CountryOfOrigin", "TariffPercentage", "NormallyStocking", "Discontinued", "EndOfLife",
            "MouserPart", "MouserPrice", "BestPrice", "BestPriceSource",
            "BestPriceLeadTime", "FastestLeadTime", "FastestLeadTimeSource", "FastestLeadTimePrice",
            "Availability"
        ]
        headings = [
            "Part Number", "Qty Needed", "Part", "Price ($)", "Min Order Qty",
            "Tariff Code", "Country of Origin", "Tariff % of Part Cost", "Normally Stocking", "Discontinued", "End of Life",
            "Mouser Part", "Mouser Price ($)", "Best Price ($)", "Best Price Source",
            "Best Price Lead Time (days)", "Fastest Lead Time (days)", "Fastest Lead Time Source",
            "Fastest Lead Time Price ($)", "Availability (DK/Mouser)"
        ]
        self.tree = ttk.Treeview(self.tree_frame, columns=columns, show="headings", height=25)
        for col, heading in zip(columns, headings):
            self.tree.heading(col, text=heading)
            self.tree.column(col, width=150 if col not in ["TariffCode"] else 200, stretch=True)
        
        self.tree_scrollbar = ttk.Scrollbar(self.tree_frame, orient="vertical", command=self.tree.yview)
        self.tree_scrollbar.pack(side="right", fill="y")
        self.tree.configure(yscrollcommand=self.tree_scrollbar.set)

        self.tree_h_scrollbar = ttk.Scrollbar(self.tree_frame, orient="horizontal", command=self.tree.xview)
        self.tree_h_scrollbar.pack(side="bottom", fill="x")
        self.tree.configure(xscrollcommand=self.tree_h_scrollbar.set)

        self.tree.pack(side="top", fill="both", expand=True)
        
        self.legend_frame = ttk.Frame(self.tree_frame)
        self.legend_frame.pack(fill="x", pady=5)
        legend_items = [
            ("Best Price", "lightgreen"),
            ("Fastest Lead Time", "lightblue"),
            ("Alternate Part", "yellow")
        ]
        for text, color in legend_items:
            ttk.Label(self.legend_frame, text=f"{text}: ", font=("Arial", 8)).pack(side="left")
            tk.Label(self.legend_frame, text="  ", background=color, width=2).pack(side="left", padx=2)

        self.analysis_frame = ttk.Frame(self.results_notebook)
        self.results_notebook.add(self.analysis_frame, text="Analysis")

        self.analysis_table_frame = ttk.Frame(self.analysis_frame)
        self.analysis_table_frame.pack(fill="x", pady=(0, 10))
        self.analysis_table = ttk.Treeview(self.analysis_table_frame, columns=["Metric", "Value"], show="headings", height=7)
        self.analysis_table.heading("Metric", text="Metric")
        self.analysis_table.heading("Value", text="Value")
        self.analysis_table.column("Metric", width=200)
        self.analysis_table.column("Value", width=200)
        self.analysis_table_scrollbar = ttk.Scrollbar(self.analysis_table_frame, orient="vertical", command=self.analysis_table.yview)
        self.analysis_table_scrollbar.pack(side="right", fill="y")
        self.analysis_table.configure(yscrollcommand=self.analysis_table_scrollbar.set)
        self.analysis_table.pack(side="top", fill="x")

        self.analysis_canvas = tk.Canvas(self.analysis_frame, bg="white")
        self.analysis_scrollbar = ttk.Scrollbar(self.analysis_frame, orient="vertical", command=self.analysis_canvas.yview)
        self.analysis_scrollable_frame = ttk.Frame(self.analysis_canvas)

        self.analysis_scrollable_frame.bind(
            "<Configure>",
            lambda e: self.analysis_canvas.configure(scrollregion=self.analysis_canvas.bbox("all"))
        )
        self.analysis_canvas.create_window((0, 0), window=self.analysis_scrollable_frame, anchor="nw")
        self.analysis_canvas.configure(yscrollcommand=self.analysis_scrollbar.set)

        self.analysis_canvas.pack(side="left", fill="both", expand=True)
        self.analysis_scrollbar.pack(side="right", fill="y")

        self.analysis_title = ttk.Label(self.analysis_scrollable_frame, text="Analysis Results", font=("TkDefaultFont", 12, "bold"))
        self.analysis_title.pack(fill="x", pady=(0, 10))

        self.ai_summary_label = ttk.Label(self.analysis_scrollable_frame, text="AI Summary: N/A", wraplength=600, background="white")
        self.ai_summary_label.pack(fill="x", pady=2)

        for entry in ["total_units", "max_premium", "target_lead_time", "cost_weight", "lead_time_weight"]:
            getattr(self, f"{entry}_entry").bind("<KeyRelease>", self.validate_inputs)

        self.bom_df = None
        self.digikey_token = None
        self.mouser_api_key = os.environ.get('MOUSER_API_KEY')
        self.mouser_requests = 0
        self.mouser_available = True
        self.part_data = []

        # Initialize Mouser API request counter
        self.mouser_request_counter_file = Path('mouser_request_counter.json')
        self.mouser_daily_limit = 1000  # Assumed daily limit
        self.mouser_requests_today = 0
        self.mouser_last_reset_date = None
        self.load_mouser_request_counter()

        self.default_tariff_rate = 0.035
        self.predictive_increase = {
            'China': 0.15, 'Mexico': 0.05, 'India': 0.10, 'Vietnam': 0.08, 'Turkey': 0.12, 'Unknown': 0.05, 'Japan': 0.05, 'Malaysia': 0.05
        }

        # HTS to country mapping for fallback
        self.hts_to_country = {
            '8532.24.0020': ['China', 'Taiwan'],  # Ceramic capacitors
            '8533.21.0030': ['Japan', 'China'],  # Resistors
            '8542.33.0001': ['Malaysia', 'China'],  # Amplifiers
            '8504.50.8000': ['Germany', 'Japan'],  # Inductors
            '8542.39.0060': ['China', 'Malaysia'],  # Integrated circuits
            '8541.21.0075': ['Taiwan', 'Philippines'],  # Transistors
            '8541.10.0070': ['South Korea', 'China'],  # Diodes
        }

        if not os.environ.get('DIGIKEY_CLIENT_ID') or not os.environ.get('DIGIKEY_CLIENT_SECRET'):
            self.status.config(text="ERROR: DIGIKEY_CLIENT_ID or DIGIKEY_CLIENT_SECRET not set", foreground="black", background="lightgray")
            logger.error("DIGIKEY_CLIENT_ID or DIGIKEY_CLIENT_SECRET not set")
        if not self.mouser_api_key:
            self.status.config(text="WARNING: MOUSER_API_KEY not set - Mouser data unavailable", foreground="black", background="lightgray")
            logger.warning("MOUSER_API_KEY not set")
            self.mouser_available = False
        else:
            logger.info(f"Using Mouser API Key: {self.mouser_api_key[:4]}...")

        if not os.environ.get('CHATGPT_API_KEY'):
            logger.warning("CHATGPT_API_KEY not set - AI analysis features will be disabled")
        else:
            logger.info("ChatGPT API key loaded successfully")

        logger.info("GUI initialization complete")

    def load_mouser_request_counter(self):
        """Load the Mouser API request counter from a file."""
        if self.mouser_request_counter_file.exists():
            try:
                with open(self.mouser_request_counter_file, 'r') as f:
                    data = json.load(f)
                last_reset_date = datetime.fromisoformat(data['last_reset_date']).date()
                today = datetime.now(timezone.utc).date()
                if last_reset_date == today:
                    self.mouser_requests_today = data['requests']
                    self.mouser_last_reset_date = last_reset_date
                else:
                    # Reset counter if it's a new day
                    self.mouser_requests_today = 0
                    self.mouser_last_reset_date = today
                    self.save_mouser_request_counter()
            except Exception as e:
                logger.error(f"Failed to load Mouser request counter: {str(e)}")
                self.mouser_requests_today = 0
                self.mouser_last_reset_date = datetime.now(timezone.utc).date()
                self.save_mouser_request_counter()
        else:
            self.mouser_requests_today = 0
            self.mouser_last_reset_date = datetime.now(timezone.utc).date()
            self.save_mouser_request_counter()

    def save_mouser_request_counter(self):
        """Save the Mouser API request counter to a file."""
        try:
            with open(self.mouser_request_counter_file, 'w') as f:
                json.dump({
                    'requests': self.mouser_requests_today,
                    'last_reset_date': self.mouser_last_reset_date.isoformat()
                }, f)
        except Exception as e:
            logger.error(f"Failed to save Mouser request counter: {str(e)}")

    def check_mouser_rate_limit(self):
        """Check if we've exceeded the Mouser API daily rate limit."""
        today = datetime.now(timezone.utc).date()
        if self.mouser_last_reset_date != today:
            self.mouser_requests_today = 0
            self.mouser_last_reset_date = today
            self.save_mouser_request_counter()

        if self.mouser_requests_today >= self.mouser_daily_limit:
            # Calculate time until reset (00:00 UTC)
            now = datetime.now(timezone.utc)
            next_reset = datetime.combine(today + timedelta(days=1), datetime.min.time(), tzinfo=timezone.utc)
            wait_seconds = (next_reset - now).total_seconds()
            logger.warning(f"Mouser API daily limit of {self.mouser_daily_limit} calls reached. Waiting {wait_seconds:.0f} seconds until reset at 00:00 UTC.")
            self.status.config(text=f"Mouser API limit reached—waiting {wait_seconds:.0f}s until reset", foreground="black", background="lightgray")
            self.root.update()
            time.sleep(wait_seconds)
            self.mouser_requests_today = 0
            self.mouser_last_reset_date = today
            self.save_mouser_request_counter()
            self.status.config(text="Mouser API limit reset, resuming...", foreground="black", background="lightgray")
            self.root.update()

    def validate_inputs(self, event=None):
        try:
            total_units = float(self.total_units_entry.get())
            max_premium = float(self.max_premium_entry.get())
            target_lead_time = float(self.target_lead_time_entry.get())
            cost_weight = float(self.cost_weight_entry.get())
            lead_time_weight = float(self.lead_time_weight_entry.get())

            if total_units <= 0:
                raise ValueError("Total Units to Build must be positive.")
            if max_premium < 0 or target_lead_time < 0 or cost_weight < 0 or lead_time_weight < 0:
                raise ValueError("Values must be non-negative.")
            if not np.isclose(cost_weight + lead_time_weight, 1.0, atol=0.01):
                raise ValueError("Cost Weight + Lead Time Weight must sum to 1.0 (within 1% tolerance).")

            self.validation_label.config(text="Valid configuration", foreground="green")
            self.run_button.config(state="normal")
            self.ai_summary_button.config(state="normal")
            return True
        except ValueError as e:
            self.validation_label.config(text=f"Invalid: {str(e)}", foreground="red")
            self.run_button.config(state="disabled")
            self.ai_summary_button.config(state="disabled")
            return False

    def load_bom(self):
        bom_path = filedialog.askopenfilename(filetypes=[("CSV Files", "*.csv")])
        if bom_path:
            try:
                self.bom_df = pd.read_csv(bom_path)
                required_cols = {"Part Number", "Quantity"}
                if not required_cols.issubset(self.bom_df.columns):
                    raise ValueError(f"BOM must contain columns: {required_cols}")
                self.file_label.config(text=f"Loaded: {bom_path} ({len(self.bom_df)} parts)")
                if len(self.bom_df) > 120:
                    self.status.config(text=f"WARNING: {len(self.bom_df)} parts > 120/min limit—batched run", foreground="black", background="lightgray")
                logger.info(f"BOM loaded: {bom_path}")
            except Exception as e:
                self.status.config(text=f"Failed to load BOM: {str(e)}", foreground="black", background="lightgray")
                logger.error(f"BOM load error: {str(e)}")

    def get_digikey_token(self):
        logger.info("Attempting to get DigiKey token...")
        CACHE_DIR = Path('./cache')
        os.makedirs(CACHE_DIR, exist_ok=True)
        token_file = CACHE_DIR / 'digikey_oauth2_token.json'
        
        try:
            with open(token_file, 'r') as f:
                self.digikey_token = json.load(f)
            logger.info("DigiKey token loaded from cache")
            return True
        except FileNotFoundError:
            logger.info("No cached DigiKey token found—triggering authentication")
            auth_url = f"https://api.digikey.com/v1/oauth2/authorize?client_id={os.environ['DIGIKEY_CLIENT_ID']}&response_type=code&redirect_uri=http://localhost:8139"
            webbrowser.open(auth_url)
            server = HTTPServer(('localhost', 8139), OAuthHandler)
            server.handle_request()
            code = server.auth_code
            if code:
                response = requests.post("https://api.digikey.com/v1/oauth2/token", data={
                    'client_id': os.environ['DIGIKEY_CLIENT_ID'],
                    'client_secret': os.environ.get('DIGIKEY_CLIENT_SECRET'),
                    'grant_type': 'authorization_code',
                    'code': code,
                    'redirect_uri': 'http://localhost:8139'
                }, headers={'Content-Type': 'application/x-www-form-urlencoded'})
                if response.status_code == 200:
                    self.digikey_token = response.json()
                    with open(token_file, 'w') as f:
                        json.dump(self.digikey_token, f)
                    logger.info("DigiKey token retrieved and cached")
                    return True
                else:
                    self.status.config(text=f"DigiKey token fetch failed: {response.status_code}", foreground="black", background="lightgray")
                    logger.error(f"DigiKey token fetch failed: {response.status_code}")
                    return False
            return False

    def refresh_digikey_token(self):
        logger.info("Attempting to refresh DigiKey token...")
        if not self.digikey_token or 'refresh_token' not in self.digikey_token:
            self.status.config(text="No refresh token available—manual reauthentication required", foreground="black", background="lightgray")
            logger.warning("No refresh token available for DigiKey")
            return False

        response = requests.post("https://api.digikey.com/v1/oauth2/token", data={
            'client_id': os.environ['DIGIKEY_CLIENT_ID'],
            'client_secret': os.environ.get('DIGIKEY_CLIENT_SECRET'),
            'grant_type': 'refresh_token',
            'refresh_token': self.digikey_token['refresh_token']
        }, headers={'Content-Type': 'application/x-www-form-urlencoded'})

        if response.status_code == 200:
            self.digikey_token = response.json()
            with open('./cache/digikey_oauth2_token.json', 'w') as f:
                json.dump(self.digikey_token, f)
            logger.info("DigiKey token refreshed successfully")
            return True
        else:
            self.status.config(text=f"Token refresh failed: {response.status_code}", foreground="black", background="lightgray")
            logger.error(f"DigiKey token refresh failed: {response.status_code}")
            return False

    def search_mouser(self, part_number):
        logger.info(f"Searching Mouser for part: {part_number}")
        if not self.mouser_available:
            logger.warning("Mouser API unavailable - skipping Mouser search")
            return None

        # Check rate limit before making the request
        self.check_mouser_rate_limit()

        url = "https://api.mouser.com/api/v1/search/keyword"
        headers = {'Accept': 'application/json', 'Content-Type': 'application/json'}
        params = {'apiKey': self.mouser_api_key}
        body = {'SearchByKeywordRequest': {'keyword': part_number, 'records': 1, 'searchOptions': ''}}

        logger.debug(f"Mouser API request: URL={url}, Headers={headers}, Params={params}, Body={body}")

        for attempt in range(3):
            try:
                # Add timeout to prevent hanging
                response = requests.post(url, headers=headers, params=params, json=body, timeout=10)
                self.mouser_requests += 1
                self.mouser_requests_today += 1  # Increment even on successful attempts
                self.save_mouser_request_counter()
                self.rate_label.config(text=f"Rate Limits - DigiKey: {self.rate_label.cget('text').split('DigiKey: ')[1].split(',')[0]}, Mouser: {self.mouser_daily_limit - self.mouser_requests_today}/{self.mouser_daily_limit}")
                
                logger.debug(f"Mouser API response status: {response.status_code}")
                logger.debug(f"Mouser API response headers: {response.headers}")
                logger.debug(f"Mouser API response body: {response.text[:500]}...")  # Truncate for brevity

                response.raise_for_status()
                data = response.json()

                if 'Errors' in data and data['Errors']:
                    error_msg = data['Errors'][0].get('Message', 'Unknown error')
                    logger.error(f"Mouser API error for {part_number}: {error_msg}")
                    if "Unauthorized" in error_msg or "Rate Limit Exceeded" in error_msg:
                        self.mouser_available = False
                        self.status.config(text="ERROR: Mouser API unauthorized or rate limit exceeded - Mouser data unavailable", foreground="black", background="lightgray")
                    return None

                parts = data.get('SearchResults', {}).get('Parts', [])
                if not parts:
                    logger.info(f"No parts found for {part_number} in Mouser")
                    return None

                part = parts[0]
                price = part.get('PriceBreaks', [{}])[0].get('Price', 'N/A')
                if isinstance(price, str) and price.startswith('$'):
                    price = price[1:]

                lead_time = part.get('LeadTime', 'N/A')
                qty_available = part.get('Availability', 'N/A').split()[0] if part.get('Availability') else 'N/A'

                try:
                    price = float(price) if price != 'N/A' else float('inf')
                except (ValueError, TypeError):
                    logger.warning(f"Invalid Mouser price for {part_number}: {price}")
                    price = float('inf')

                try:
                    qty_available = int(qty_available) if qty_available != 'N/A' else 0
                except (ValueError, TypeError):
                    logger.warning(f"Invalid Mouser quantity for {part_number}: {qty_available}")
                    qty_available = 0

                result = {
                    "MPN": part.get('ManufacturerPartNumber', 'N/A'),
                    "Manufacturer": part.get('Manufacturer', 'N/A'),
                    "Description": part.get('Description', 'N/A'),
                    "Price": price,
                    "SourcePartNumber": part.get('MouserPartNumber', 'N/A'),
                    "Source": "Mouser",
                    "QtyAvailable": qty_available,
                    "LeadTime": lead_time,
                    "DatasheetUrl": part.get('DataSheetUrl', 'N/A')
                }
                logger.debug(f"Mouser API result for {part_number}: {result}")
                return result

            except requests.Timeout:
                logger.error(f"Mouser API request timed out for {part_number} after 10 seconds")
                self.mouser_requests_today += 1  # Increment on timeout attempt
                self.save_mouser_request_counter()
                self.status.config(text=f"Mouser API timed out for {part_number}, retrying...", foreground="black", background="lightgray")
                self.root.update()
                if attempt < 2:
                    logger.info(f"Retrying ({attempt + 2}/3) after 5s...")
                    time.sleep(5)
                    continue
                self.status.config(text=f"Mouser API timeout for {part_number}, proceeding without Mouser data", foreground="black", background="lightgray")
                self.root.update()
                return None
            except requests.RequestException as e:
                logger.error(f"Mouser API request failed for {part_number}: {str(e)}")
                self.mouser_requests_today += 1  # Increment on failed attempt
                self.save_mouser_request_counter()
                if attempt < 2:
                    logger.info(f"Retrying ({attempt + 2}/3) after 5s...")
                    time.sleep(5)
                    continue
                self.status.config(text=f"Mouser API failed for {part_number}, proceeding without Mouser data", foreground="black", background="lightgray")
                self.root.update()
                return None

    def convert_lead_time_to_days(self, lead_time):
        logger.debug(f"Converting lead time: {lead_time}")
        if lead_time == 'N/A':
            return float('inf')
        try:
            if isinstance(lead_time, str):
                days = int(lead_time.split()[0])
                return days
            return int(lead_time) * 7
        except (ValueError, IndexError):
            logger.warning(f"Failed to convert lead time: {lead_time}")
            return float('inf')

    def get_optimal_price(self, qty_needed, pricing_breaks, min_order_qty):
        logger.debug(f"Calculating optimal price: qty_needed={qty_needed}, min_order_qty={min_order_qty}")
        if not pricing_breaks:
            logger.warning("No pricing breaks provided, returning infinity")
            return float('inf'), float('inf')
        qty_to_order = max(qty_needed, min_order_qty)
        pricing_breaks.sort(key=lambda x: x["BreakQuantity"])
        unit_price = float('inf')
        for break_point in pricing_breaks:
            if qty_to_order >= break_point["BreakQuantity"]:
                unit_price = break_point["UnitPrice"]
            else:
                break
        if unit_price == float('inf') and pricing_breaks:
            unit_price = pricing_breaks[-1]["UnitPrice"]
        total_price = unit_price * qty_to_order
        logger.debug(f"Optimal price calculated: unit_price={unit_price}, total_price={total_price}")
        return unit_price, total_price

    def validate_and_run(self):
        logger.info("validate_and_run called")
        is_valid = self.validate_inputs()
        if is_valid:
            self.run_analysis()
        else:
            logger.warning("Inputs invalid, not running analysis")

    def infer_country_from_hts(self, hts_code):
        countries = self.hts_to_country.get(hts_code, None)
        if countries:
            return countries[0]  # Return the first likely country
        return "Unknown"

    def run_analysis(self):
        logger.info("run_analysis called")
        if self.bom_df is None or self.bom_df.empty:
            self.status.config(text="Load a BOM first!", foreground="black", background="lightgray")
            logger.warning("No BOM loaded, exiting run_analysis")
            return
        if not self.get_digikey_token():
            logger.error("Failed to get DigiKey token, exiting run_analysis")
            return

        self.status.config(text="Running...", foreground="black", background="lightgray")
        self.progress.grid(row=1, column=0, columnspan=6, padx=5, pady=2)
        self.progress["maximum"] = len(self.bom_df)
        self.progress["value"] = 0

        for item in self.tree.get_children():
            self.tree.delete(item)
        for item in self.analysis_table.get_children():
            self.analysis_table.delete(item)

        try:
            total_units = float(self.total_units_entry.get())
            max_premium = float(self.max_premium_entry.get()) / 100
            target_lead_time = float(self.target_lead_time_entry.get()) * 7
            cost_weight = float(self.cost_weight_entry.get())
            lead_time_weight = float(self.lead_time_weight_entry.get())
            custom_tariff_rates = {country: float(self.tariff_entries[country].get() or "0") / 100 for country in self.tariff_entries}
            # Treat 0.0 as "use default" instead of forcing 0%
            custom_tariff_rates = {country: rate if rate > 0 else None for country, rate in custom_tariff_rates.items()}
            logger.info(f"Sweet spot parameters: total_units={total_units}, max_premium={max_premium}, target_lead_time={target_lead_time}, cost_weight={cost_weight}, lead_time_weight={lead_time_weight}, custom_tariff_rates={custom_tariff_rates}")
        except ValueError as e:
            self.status.config(text=f"Invalid sweet spot parameters: {str(e)}", foreground="black", background="lightgray")
            self.progress.grid_forget()
            logger.error(f"Invalid sweet spot parameters: {str(e)}")
            return

        digikey_headers = {
            'Authorization': f"Bearer {self.digikey_token['access_token']}",
            'X-DIGIKEY-Client-Id': os.environ['DIGIKEY_CLIENT_ID'],
            'X-DIGIKEY-Locale-Site': 'US',
            'X-DIGIKEY-Locale-Language': 'en',
            'X-DIGIKEY-Locale-Currency': 'USD',
            'X-DIGIKEY-Locale-ShipToCountry': 'us',
            'X-DIGIKEY-Customer-Id': '0',
            'Content-Type': 'application/json'
        }
        digikey_url = "https://api.digikey.com/products/v4/search/keyword"
        results = []
        request_count = 0
        lead_times = []
        self.part_data = []

        # Process DigiKey requests sequentially
        parts = self.bom_df['Part Number'].tolist()
        quantities = self.bom_df['Quantity'].tolist()
        total_parts = len(parts)

        for i, (part, qty) in enumerate(zip(parts, quantities)):
            qty = qty * total_units
            logger.info(f"Processing part: {part}")

            # Update progress
            self.progress["value"] = i + 1
            percentage = (i + 1) / total_parts * 100
            self.progress_label.config(text=f"Progress: {percentage:.1f}% - Processing {part}")
            self.root.update_idletasks()

            burst_remaining = int(self.rate_label.cget("text").split("DigiKey: ")[1].split("/")[0]) if request_count > 0 else 120
            if burst_remaining < 10:
                pause_time = 60 - (time.time() % 60)
                self.status.config(text=f"Low burst limit—pausing {pause_time:.0f}s...", foreground="black", background="lightgray")
                self.root.update()
                time.sleep(pause_time)
                request_count = 0

            dk_response = requests.post(digikey_url, headers=digikey_headers, json={"Keywords": part, "Limit": 1, "Offset": 0})
            request_count += 1

            if dk_response.status_code == 401:
                if self.refresh_digikey_token():
                    digikey_headers['Authorization'] = f"Bearer {self.digikey_token['access_token']}"
                    dk_response = requests.post(digikey_url, headers=digikey_headers, json={"Keywords": part, "Limit": 1, "Offset": 0})
                else:
                    self.status.config(text="Token refresh failed—please reauthenticate manually", foreground="black", background="lightgray")
                    logger.error("Token refresh failed, exiting run_analysis")
                    return

            burst_remaining = dk_response.headers.get('X-BurstLimit-Remaining', '120')
            self.rate_label.config(text=f"Rate Limits - DigiKey: {burst_remaining}/120, Mouser: {self.mouser_daily_limit - self.mouser_requests_today}/{self.mouser_daily_limit}")

            # Parallelize Mouser requests with timeout
            mouser_result = None
            with ThreadPoolExecutor(max_workers=5) as executor:
                future = executor.submit(self.search_mouser, part)
                try:
                    mouser_result = future.result(timeout=15)  # 15-second timeout
                except TimeoutError:
                    logger.error(f"Mouser API request for {part} timed out after 15 seconds")
                    executor._threads.clear()  # Clear threads to prevent hanging
                except Exception as e:
                    logger.error(f"Mouser API request for {part} failed: {str(e)}")
                    executor._threads.clear()

            # Initialize result with default values
            result = {
                "PartNumber": part,
                "QtyNeeded": qty,
                "Part": "N/A",
                "Price": "N/A",
                "MinOrderQty": "N/A",
                "TariffCode": "N/A",
                "CountryOfOrigin": "N/A",
                "TariffPercentage": "N/A",
                "NormallyStocking": "N/A",
                "Discontinued": "N/A",
                "EndOfLife": "N/A",
                "MouserPart": mouser_result["MPN"] if mouser_result else "N/A",
                "MouserPrice": f"{mouser_result['Price']:.2f}" if mouser_result and mouser_result.get('Price') != float('inf') else "N/A",
                "BestPrice": "N/A",
                "BestPriceSource": "N/A",
                "BestPriceLeadTime": "N/A",
                "FastestLeadTime": "N/A",
                "FastestLeadTimeSource": "N/A",
                "FastestLeadTimePrice": "N/A",
                "Availability": "N/A"
            }

            dk_price = float('inf')
            dk_total_price = float('inf')
            dk_lead_time = float('inf')
            dk_qty_available = 0
            dk_min_order_qty = 0
            dk_part_number = "N/A"
            dk_normally_stocking = False
            dk_discontinued = False
            dk_end_of_life = False
            dk_hts_code = "N/A"
            mouser_price = float('inf') if not mouser_result else mouser_result.get('Price', float('inf'))
            mouser_lead_time = float('inf') if not mouser_result else self.convert_lead_time_to_days(mouser_result.get('LeadTime', 'N/A'))
            mouser_qty_available = 0 if not mouser_result else mouser_result.get('QtyAvailable', 0)
            mouser_part_number = "N/A" if not mouser_result else mouser_result.get('MPN', 'N/A')

            if dk_response.status_code == 200:
                dk_data = dk_response.json()
                if dk_data.get("Products"):
                    product = dk_data["Products"][0]
                    dk_qty_available = product.get("QuantityAvailable", 0)
                    dk_lead_time = self.convert_lead_time_to_days(product.get("ManufacturerLeadWeeks", "N/A"))
                    dk_part_number = product.get("ManufacturerProductNumber", "N/A")
                    dk_min_order_qty = product.get("MinimumOrderQuantity", 0)
                    dk_normally_stocking = product.get("NormallyStocking", False)
                    dk_discontinued = product.get("Discontinued", False)
                    dk_end_of_life = product.get("EndOfLife", False)
                    dk_hts_code = product.get("Classifications", {}).get("HtsusCode", "N/A")

                    dk_price_raw = product.get("UnitPrice", None)
                    dk_price = float(dk_price_raw) if dk_price_raw is not None else float('inf')

                    dk_price_opt, dk_total_price = self.get_optimal_price(qty, product.get("StandardPricing", []), dk_min_order_qty)

                    if dk_price != float('inf'):
                        dk_price = dk_price_raw
                    elif dk_price_opt != float('inf'):
                        dk_price = dk_price_opt

                    result["Part"] = dk_part_number
                    result["Price"] = f"{dk_price:.2f}" if dk_price != float('inf') else "N/A"
                    result["MinOrderQty"] = str(dk_min_order_qty) if dk_min_order_qty is not None else "N/A"
                    result["TariffCode"] = dk_hts_code
                    result["NormallyStocking"] = str(dk_normally_stocking)
                    result["Discontinued"] = str(dk_discontinued)
                    result["EndOfLife"] = str(dk_end_of_life)
                    if dk_qty_available > 0:
                        lead_times.append(dk_lead_time)
            elif dk_response.status_code == 429:
                retry_after = int(dk_response.headers.get('Retry-After', 60))
                self.status.config(text=f"429 - Pausing {retry_after}s...", foreground="black", background="lightgray")
                self.root.update()
                time.sleep(retry_after)
                request_count -= 1
                continue

            # Determine best price and update result
            if mouser_result and mouser_price < dk_price and mouser_qty_available > 0:
                result["Price"] = f"{mouser_price:.2f}"
                result["BestPrice"] = f"{mouser_price:.2f}"
                result["BestPriceSource"] = "Mouser"
                result["BestPriceLeadTime"] = f"{mouser_lead_time:.0f}" if mouser_lead_time != float('inf') else "N/A"
                result["Availability"] = f"{dk_qty_available}/{mouser_qty_available}"
            elif dk_price != float('inf') and dk_qty_available > 0:
                result["BestPrice"] = f"{dk_price:.2f}"
                result["BestPriceSource"] = "DigiKey"
                result["BestPriceLeadTime"] = f"{dk_lead_time:.0f}" if dk_lead_time != float('inf') else "N/A"
                result["Availability"] = f"{dk_qty_available}/0"
            else:
                result["Availability"] = "0/0"

            # Determine fastest lead time
            fastest_lead_time = min(dk_lead_time, mouser_lead_time) if mouser_lead_time != float('inf') else dk_lead_time
            fastest_lead_time_source = "DigiKey" if dk_lead_time <= mouser_lead_time else "Mouser"
            fastest_lead_time_price = dk_price if dk_lead_time <= mouser_lead_time else mouser_price

            result["FastestLeadTime"] = f"{fastest_lead_time:.0f}" if fastest_lead_time != float('inf') else "N/A"
            result["FastestLeadTimeSource"] = fastest_lead_time_source
            result["FastestLeadTimePrice"] = f"{fastest_lead_time_price:.2f}" if fastest_lead_time_price != float('inf') else "N/A"

            # Debug the result dictionary before inserting into table
            logger.debug(f"Result dictionary before table insert: {result}")

            part_data = {
                "part": part,
                "qty": qty,
                "original_qty": self.bom_df[self.bom_df['Part Number'] == part]['Quantity'].iloc[0],
                "dk_part": dk_part_number,
                "dk_price": dk_price,
                "dk_total_price": dk_total_price,
                "dk_lead_time": dk_lead_time,
                "dk_qty_available": dk_qty_available,
                "dk_min_order_qty": dk_min_order_qty,
                "dk_normally_stocking": dk_normally_stocking,
                "dk_discontinued": dk_discontinued,
                "dk_end_of_life": dk_end_of_life,
                "dk_hts_code": dk_hts_code,
                "mouser_part": mouser_part_number,
                "mouser_price": mouser_price,
                "mouser_lead_time": mouser_lead_time,
                "mouser_qty_available": mouser_qty_available
            }
            self.part_data.append(part_data)

            item_id = self.tree.insert("", "end", values=tuple(result.values()))

        # Perform tariff analysis once for all parts
        if os.environ.get('CHATGPT_API_KEY'):
            try:
                lead_time_data = "\n".join(
                    f"- Part {data['part']}: DigiKey {data['dk_lead_time']} days, Mouser {data['mouser_lead_time']} days"
                    for data in self.part_data
                )
                lead_time_prompt = f"""
                Given the following lead time data for electronic components (in days):
                {lead_time_data}
                Analyze this data to:
                1. Identify industry-specific trends in lead times.
                2. Detect any seasonal variations.
                3. Flag components with historically volatile lead times.
                4. Assign a risk score (0-100) to each component based on lead time volatility.
                Provide a detailed analysis.
                """
                lead_time_analysis = call_chatgpt(lead_time_prompt) or "Lead time analysis unavailable due to API error."

                component_data = "\n".join(
                    f"- {data['part']}: {data['dk_part'] if data['dk_part'] != 'N/A' else data['mouser_part']} (HTS: {data['dk_hts_code']})"
                    for data in self.part_data
                )
                tariff_prompt = f"""
                Given the following components, their manufacturers, and HTS codes:
                {component_data}
                Perform a tariff assessment:
                1. Determine the likely country of origin for each component using HTS codes, manufacturer patterns, and historical market data.
                2. Calculate a country of origin diversity score (0-100).
                3. Recommend alternative sourcing to mitigate tariff risks.
                4. Estimate the cost impact of potential new tariffs (assume a 10% tariff increase).
                5. Highlight regulatory compliance considerations by region (e.g., US, EU).
                Provide a detailed analysis.
                """
                tariff_analysis = call_chatgpt(tariff_prompt)

                country_data = {}
                if tariff_analysis:
                    for line in tariff_analysis.split('\n'):
                        if 'is likely manufactured in' in line.lower():
                            parts = line.split(':')
                            if len(parts) > 1:
                                part_info = parts[0].strip().replace('-', '').strip()
                                country_part = parts[1].strip()
                                country = country_part.split('is likely manufactured in')[1].strip() if 'is likely manufactured in' in country_part else "Unknown"
                                country_data[part_info] = country if country else "Unknown"
                else:
                    logger.warning("ChatGPT tariff analysis failed, using HTS fallback")

                # Apply tariff calculations to all parts
                for item_id in self.tree.get_children():
                    values = list(self.tree.item(item_id, 'values'))
                    part = values[0]  # PartNumber
                    price = float(values[3].replace("N/A", "0")) if values[3] != "N/A" else float(values[12].replace("N/A", "0")) if values[12] != "N/A" else 0.0
                    qty = float(values[1])
                    tariff_code = values[5]

                    country = country_data.get(part, "Unknown")
                    if country == "Unknown":
                        hts_code = tariff_code
                        if hts_code and hts_code != "N/A":
                            country = self.infer_country_from_hts(hts_code)
                        country_data[part] = country

                    custom_tariff_rate = custom_tariff_rates.get(country)
                    if custom_tariff_rate is not None:
                        base_tariff_rate = custom_tariff_rate
                        predictive_tariff_rate = custom_tariff_rate + 0.10
                    else:
                        if tariff_code and tariff_code != 'N/A':
                            tariff_rate = self.fetch_usitc_tariff_rate(tariff_code)
                            logger.debug(f"USITC tariff rate for HTS {tariff_code}: {tariff_rate}")
                            if tariff_rate is not None:
                                base_tariff_rate = tariff_rate
                            else:
                                base_tariff_rate = self.predictive_increase.get(country, self.default_tariff_rate) if country != 'United States' else 0.0
                        else:
                            base_tariff_rate = self.predictive_increase.get(country, self.default_tariff_rate) if country != 'United States' else 0.0
                        predictive_tariff_rate = base_tariff_rate + self.predictive_increase.get(country, 0.05)

                    part_tariff_cost = price * qty * base_tariff_rate if base_tariff_rate is not None else 0.0
                    part_total_cost = price * qty
                    tariff_percentage = (part_tariff_cost / part_total_cost * 100) if part_total_cost > 0 else 0.0

                    logger.info(f"Part {part}: Country={country}, TariffCode={tariff_code}, Base Tariff={base_tariff_rate*100 if base_tariff_rate is not None else 'N/A'}%, Predictive Tariff={predictive_tariff_rate*100 if predictive_tariff_rate is not None else 'N/A'}%, Tariff Cost=${part_tariff_cost:.2f}, Tariff %={tariff_percentage:.2f}%")

                    values[6] = country  # CountryOfOrigin
                    values[7] = f"{tariff_percentage:.2f}%" if tariff_percentage > 0 else "N/A"  # TariffPercentage
                    self.tree.item(item_id, values=tuple(values))

            except Exception as e:
                logger.error(f"Error processing tariff analysis: {str(e)}")
                for item_id in self.tree.get_children():
                    values = list(self.tree.item(item_id, 'values'))
                    part = values[0]
                    hts_code = values[5]
                    country = self.infer_country_from_hts(hts_code) if hts_code and hts_code != "N/A" else "Unknown"
                    price = float(values[3].replace("N/A", "0")) if values[3] != "N/A" else float(values[12].replace("N/A", "0")) if values[12] != "N/A" else 0.0
                    qty = float(values[1])
                    tariff_code = values[5]

                    custom_tariff_rate = custom_tariff_rates.get(country)
                    if custom_tariff_rate is not None:
                        base_tariff_rate = custom_tariff_rate
                        predictive_tariff_rate = custom_tariff_rate + 0.10
                    else:
                        base_tariff_rate = self.predictive_increase.get(country, self.default_tariff_rate) if country != 'United States' else 0.0
                        predictive_tariff_rate = base_tariff_rate + self.predictive_increase.get(country, 0.05)

                    part_tariff_cost = price * qty * base_tariff_rate if base_tariff_rate is not None else 0.0
                    part_total_cost = price * qty
                    tariff_percentage = (part_tariff_cost / part_total_cost * 100) if part_total_cost > 0 else 0.0

                    values[6] = country
                    values[7] = f"{tariff_percentage:.2f}%" if tariff_percentage > 0 else "N/A"
                    self.tree.item(item_id, values=tuple(values))

        self.tree.update_idletasks()
        self.root.update()

        if lead_times:
            lowest_lead_time = min(lead_times)
            self.analysis_table.insert("", "end", values=("Lowest Lead Time", f"{lowest_lead_time} days"))
        else:
            self.analysis_table.insert("", "end", values=("Lowest Lead Time", "N/A (No stock data)"))

        total_costs_min = [min(d["dk_total_price"], d["mouser_price"] * d["qty"]) for d in self.part_data if min(d["dk_total_price"], d["mouser_price"]) != float('inf')]
        total_costs_max = [max(d["dk_total_price"], d["mouser_price"] * d["qty"]) for d in self.part_data if max(d["dk_total_price"], d["mouser_price"]) != float('inf')]
        if total_costs_min and total_costs_max:
            total_min_cost = sum(total_costs_min)
            total_max_cost = sum(total_costs_max)
            self.analysis_table.insert("", "end", values=("Min/Max Cost", f"${total_min_cost:.2f} / ${total_max_cost:.2f}"))
        else:
            self.analysis_table.insert("", "end", values=("Min/Max Cost", "N/A (No cost data)"))

        cheapest_cost = 0
        cheapest_max_lead_time = 0
        for data in self.part_data:
            min_cost = min(data["dk_total_price"], data["mouser_price"] * data["qty"])
            source = "DigiKey" if data["dk_total_price"] <= data["mouser_price"] * data["qty"] else "Mouser"
            lead_time = data["dk_lead_time"] if source == "DigiKey" else data["mouser_lead_time"]
            if min_cost != float('inf'):
                cheapest_cost += min_cost
            if lead_time != float('inf'):
                cheapest_max_lead_time = max(cheapest_max_lead_time, lead_time)
        self.analysis_table.insert("", "end", values=("Cheapest Option", f"${cheapest_cost:.2f}, Max Lead Time: {cheapest_max_lead_time:.0f} days"))

        cheapest_stock_cost = 0
        cheapest_stock_max_lead_time = 0
        clear_to_build_issues = []
        for data in self.part_data:
            if data["dk_qty_available"] >= data["qty"] and (data["mouser_qty_available"] < data["qty"] or data["dk_total_price"] <= data["mouser_price"] * data["qty"]):
                cheapest_stock_cost += data["dk_total_price"]
                cheapest_stock_max_lead_time = max(cheapest_stock_max_lead_time, data["dk_lead_time"])
            elif data["mouser_qty_available"] >= data["qty"]:
                cheapest_stock_cost += data["mouser_price"] * data["qty"]
                cheapest_stock_max_lead_time = max(cheapest_stock_max_lead_time, data["mouser_lead_time"])
            else:
                source = "DigiKey" if data["dk_lead_time"] <= data["mouser_lead_time"] else "Mouser"
                lead_time = data["dk_lead_time"] if source == "DigiKey" else data["mouser_lead_time"]
                clear_to_build_issues.append(f"Part {data['part']}: Required {data['qty']}, Available {data['dk_qty_available']}/{data['mouser_qty_available']}, Lead Time {lead_time:.0f} days")
        if cheapest_stock_cost > 0:
            stock_message = f"${cheapest_stock_cost:.2f}, Max Lead Time: {cheapest_stock_max_lead_time:.0f} days"
            if clear_to_build_issues:
                stock_message += "\nClear-to-Build Issues:\n" + "\n".join(clear_to_build_issues)
            self.analysis_table.insert("", "end", values=("Cheapest Option with Stock", stock_message))
        else:
            stock_message = "N/A (No stock available)"
            if clear_to_build_issues:
                stock_message += "\nClear-to-Build Issues:\n" + "\n".join(clear_to_build_issues)
            self.analysis_table.insert("", "end", values=("Cheapest Option with Stock", stock_message))

        cheapest_lead_cost = 0
        cheapest_lead_max_lead_time = 0
        for data in self.part_data:
            valid_options = []
            if data["dk_total_price"] != float('inf') and data["dk_lead_time"] != float('inf'):
                valid_options.append((data["dk_total_price"], data["dk_lead_time"]))
            if data["mouser_price"] != float('inf') and data["mouser_lead_time"] != float('inf'):
                valid_options.append((data["mouser_price"] * data["qty"], data["mouser_lead_time"]))
            if valid_options:
                price, lead_time = min(valid_options, key=lambda x: x[0])
                cheapest_lead_cost += price
                cheapest_lead_max_lead_time = max(cheapest_lead_max_lead_time, lead_time)
        if cheapest_lead_cost > 0:
            self.analysis_table.insert("", "end", values=("Cheapest Option with Lead Time", f"${cheapest_lead_cost:.2f}, Max Lead Time: {cheapest_lead_max_lead_time:.0f} days"))
        else:
            self.analysis_table.insert("", "end", values=("Cheapest Option with Lead Time", "N/A (No lead time data)"))

        fastest_cost = 0
        fastest_max_lead_time = float('inf')
        for data in self.part_data:
            min_lead_time = min(data["dk_lead_time"], data["mouser_lead_time"])
            source = "DigiKey" if data["dk_lead_time"] <= data["mouser_lead_time"] else "Mouser"
            price = data["dk_total_price"] if source == "DigiKey" else data["mouser_price"] * data["qty"]
            if price != float('inf'):
                fastest_cost += price
            if min_lead_time != float('inf'):
                fastest_max_lead_time = min(fastest_max_lead_time, min_lead_time)
        self.analysis_table.insert("", "end", values=("Fastest Option", f"${fastest_cost:.2f}, Max Lead Time: {fastest_max_lead_time:.0f} days"))

        baseline_cost = cheapest_cost
        baseline_lead_time = cheapest_max_lead_time
        best_cost = float('inf')
        best_lead_time = float('inf')
        best_score = float('inf')
        best_strategy = []

        for data in self.part_data:
            scenarios = []
            if data["dk_total_price"] != float('inf') and data["dk_qty_available"] > 0:
                penalty = 0
                if not data["dk_normally_stocking"]:
                    penalty += 0.1
                if data["dk_discontinued"]:
                    penalty += 0.2
                if data["dk_end_of_life"]:
                    penalty += 0.3
                scenarios.append((data["dk_total_price"], data["dk_lead_time"], "DigiKey", penalty))
            if data["mouser_price"] != float('inf') and data["mouser_qty_available"] > 0:
                scenarios.append((data["mouser_price"] * data["qty"], data["mouser_lead_time"], "Mouser", 0))

            for cost, lead_time, source, penalty in scenarios:
                current_strategy = [(d["part"], "DigiKey" if d["dk_total_price"] <= d["mouser_price"] * d["qty"] and d["dk_qty_available"] > 0 else "Mouser") for d in self.part_data if d != data]
                current_strategy.append((data["part"], source))
                
                total_cost = sum(
                    d["qty"] * (d["dk_total_price"] if (d["part"], "DigiKey") in current_strategy and d["dk_qty_available"] > 0 else d["mouser_price"] * d["qty"])
                    for d in self.part_data if (d["dk_total_price"] != float('inf') and d["dk_qty_available"] > 0) or (d["mouser_price"] != float('inf') and d["mouser_qty_available"] > 0)
                )
                max_lead_time = max(
                    d["dk_lead_time"] if (d["part"], "DigiKey") in current_strategy else d["mouser_lead_time"]
                    for d in self.part_data if d["dk_lead_time"] != float('inf') or d["mouser_lead_time"] != float('inf')
                )
                
                premium = (total_cost - baseline_cost) / baseline_cost if baseline_cost > 0 else float('inf')
                if premium <= max_premium and max_lead_time <= target_lead_time:
                    normalized_cost = (total_cost - baseline_cost) / (fastest_cost - baseline_cost) if fastest_cost != baseline_cost else 0
                    normalized_lead_time = max_lead_time / baseline_lead_time if baseline_lead_time > 0 else 0
                    score = cost_weight * normalized_cost + lead_time_weight * normalized_lead_time + penalty
                    if score < best_score:
                        best_score = score
                        best_cost = total_cost
                        best_lead_time = max_lead_time
                        best_strategy = current_strategy

        if best_cost != float('inf'):
            self.analysis_table.insert("", "end", values=("Sweet Spot", f"${best_cost:.2f}, Max Lead Time: {best_lead_time:.0f} days"))
        else:
            self.analysis_table.insert("", "end", values=("Sweet Spot", "N/A (No viable strategy within constraints)"))

        if lead_times:
            lead_times = [lt for lt in lead_times if lt != float('inf')]
            if lead_times:
                avg_lead_time = np.mean(lead_times)
                median_lead_time = np.median(lead_times)
                self.analysis_table.insert("", "end", values=("Lead Time Stats", f"Avg {avg_lead_time:.1f} days, Median {median_lead_time:.1f} days"))
            else:
                self.analysis_table.insert("", "end", values=("Lead Time Stats", "N/A (No valid lead times)"))
        else:
            self.analysis_table.insert("", "end", values=("Lead Time Stats", "N/A (No data)"))

        # Calculate total tariff cost and percentage
        total_tariff_cost = 0
        total_bom_cost = 0
        for item_id in self.tree.get_children():
            values = self.tree.item(item_id, 'values')
            price = float(values[3].replace("N/A", "0")) if values[3] != "N/A" else float(values[12].replace("N/A", "0")) if values[12] != "N/A" else 0.0
            qty = float(values[1])
            tariff_percentage = float(values[7].replace("%", "").replace("N/A", "0")) if values[7] != "N/A" else 0.0
            part_total_cost = price * qty
            total_bom_cost += part_total_cost
            total_tariff_cost += (tariff_percentage / 100) * part_total_cost

        total_tariff_percentage = (total_tariff_cost / total_bom_cost * 100) if total_bom_cost > 0 else 0.0
        self.analysis_table.insert("", "end", values=("Total Tariff Cost", f"${total_tariff_cost:.2f}"))
        self.analysis_table.insert("", "end", values=("Total Tariff % of BOM Cost", f"{total_tariff_percentage:.2f}%"))

        if os.environ.get('CHATGPT_API_KEY'):
            pricing_data = "\n".join(
                f"- {data['part']}: DigiKey ${data['dk_price']:.2f}, Mouser ${data['mouser_price']:.2f}"
                for data in self.part_data
            )
            pricing_prompt = f"""
            Given the following pricing data for electronic components (in USD):
            {pricing_data}
            Perform a predictive pricing analysis:
            1. Visualize price trends (describe a trend line).
            2. Analyze seasonality in pricing.
            3. Develop a risk-adjusted future pricing model (predict prices for the next 6 months).
            4. Notify if any components are trending toward significant price changes (>20%).
            Provide a detailed analysis.
            """
            pricing_analysis = call_chatgpt(pricing_prompt) or "Pricing analysis unavailable due to API error."

            predictive_tariff_cost = total_tariff_cost * 1.10  # 10% increase
            tariff_output = f"Tariff Assessment:\n{tariff_analysis if tariff_analysis else 'Tariff analysis unavailable.'}\n\nTotal Possible Tariff Cost: ${total_tariff_cost:.2f}\nPredictive Tariff Cost: ${predictive_tariff_cost:.2f}\nTotal Tariff % of BOM Cost: {total_tariff_percentage:.2f}%\n\nDisclaimer: The 'likely manufactured in' results and tariff rates are estimations based on AI analysis and must be confirmed with suppliers when ordering."

            self.ai_summary_label.config(text=f"{tariff_output}\n{lead_time_analysis}\n{pricing_analysis}")

        self.status.config(text="Done!", foreground="black", background="lightgray")
        self.progress.grid_forget()
        self.progress_label.config(text="Progress: 100% - Done!")
        self.root.update_idletasks()
        with open('bom_results.json', 'w') as f:
            json.dump(results, f, indent=4)
        logger.info("Analysis complete")

    def generate_ai_summary(self):
        logger.info("Generating AI summary...")
        if not self.part_data:
            self.ai_summary_label.config(text="AI Summary: No data available. Run analysis first.")
            return

        summary = "AI Summary of Recommendations (Grok 3):\n"
        summary += f"- Based on current data, the 'Sweet Spot' option is recommended with a total cost of ${float(self.analysis_table.item(self.analysis_table.get_children()[-4], 'values')[1].split('$')[1].split(',')[0]):.2f} and a max lead time of {float(self.analysis_table.item(self.analysis_table.get_children()[-4], 'values')[1].split('Max Lead Time: ')[1].split(' days')[0]):.0f} days.\n"
        summary += f"- Cheapest Option: ${float(self.analysis_table.item(self.analysis_table.get_children()[2], 'values')[1].split('$')[1].split(',')[0]):.2f} with a max lead time of {float(self.analysis_table.item(self.analysis_table.get_children()[2], 'values')[1].split('Max Lead Time: ')[1].split(' days')[0]):.0f} days.\n"
        summary += f"- Fastest Option: ${float(self.analysis_table.item(self.analysis_table.get_children()[5], 'values')[1].split('$')[1].split(',')[0]):.2f} with a max lead time of {float(self.analysis_table.item(self.analysis_table.get_children()[5], 'values')[1].split('Max Lead Time: ')[1].split(' days')[0]):.0f} days.\n"
        summary += "- Notes: Optimized pricing uses minimum order quantities and bulk discounts where applicable.\n"
        summary += "- Warnings: Some parts may not be normally stocked, discontinued, or nearing end-of-life, affecting reliability.\n"
        summary += "- Incomplete Calculations: Tariff Risk, EOL Risk, and Future Cost Prediction are placeholders and should be integrated for a complete analysis.\n"
        summary += "- Recommendation: Prioritize 'Sweet Spot' unless speed is critical (choose 'Fastest Option') or cost is the only factor (choose 'Cheapest Option'). Review discontinued/end-of-life parts for alternatives."

        self.ai_summary_label.config(text=summary)
        logger.info("AI summary generated")

    def fetch_usitc_tariff_rate(self, hts_code):
        if not hts_code or hts_code == "N/A":
            return None
        
        try:
            search_url = f"https://hts.usitc.gov/reststop/search?keyword={hts_code}"
            response = requests.get(search_url, timeout=10)
            response.raise_for_status()
            data = response.json()

            if data and isinstance(data, list) and len(data) > 0:
                article = data[0]
                tariff_rate = article.get('general_rate', None)
                if tariff_rate and isinstance(tariff_rate, str):
                    tariff_rate = float(tariff_rate.replace('%', '')) / 100
                return tariff_rate
            else:
                logger.warning(f"No tariff data found for HTS code {hts_code}")
                return None

        except requests.RequestException as e:
            logger.error(f"Failed to fetch USITC tariff rate for HTS {hts_code}: {str(e)}")
            return None
        except (ValueError, KeyError) as e:
            logger.error(f"Error parsing USITC tariff response for HTS {hts_code}: {str(e)}")
            return None

if __name__ == "__main__":
    root = tk.Tk()
    app = BOMAnalyzerApp(root)
    app.validate_inputs()
    logger.info("Starting mainloop...")
    root.mainloop()